In [1]:
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()
from matplotlib import pyplot as plt, cm, colors
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D

sns.set_style("ticks")
sns.set_style("ticks")
sns.despine()
from matplotlib import pyplot as plt, cm, colors
from tqdm.notebook import tqdm

import glob
import os.path as ospath
import os
import pickle
import re
from sys import executable
from subprocess import check_output
from PyQt5.QtWidgets import QFileDialog, QApplication
from IPython.display import HTML

from scipy import optimize
from scipy.spatial import distance
from scipy import linalg
from scipy import signal
from scipy import stats
from sklearn.cluster import MeanShift, estimate_bandwidth

from picasso.picasso import io
from picasso.picasso.postprocess import link, compute_dark_times
from picasso.picasso.render import render
from picasso.picasso.gui.render import estimate_kinetic_rate, fit_cum_exp
from picasso.picasso.lib import append_to_rec



<Figure size 640x480 with 0 Axes>

In [3]:
# define visualization options 
%matplotlib inline
%gui qt

# define colors:
blue = "#4C72B0"
orange = "#DD8452"
red = "#C44E52"
gray = "#90A8CE"


def OpenFileDialog():
    file = check_output([executable, __file__])
    return file.strip()


def gui_fname(dir=None):
    """
    Select a file via a dialog and return the file name.
    """
    if dir is None: 
        dir ="./"

    app = QApplication([dir])
    fname = QFileDialog.getExistingDirectory(None, "Select a folder...", 
            dir)
    if isinstance(fname, tuple):
        return fname[0]
    else: 
        return str(fname)


def load_files(dirname):
    
    os.chdir(dirname)
    files = glob.glob("*.hdf5")
    
    if files:
        print("{} HDF5 files found.".format(len(files)))
    else:
        print("No HDF5 files found at: {}".format(dirname))
            
    return files


def load_ring_data_df(dirname, filename):
    file = ospath.join(dirname, filename)

    try: 
        df = pd.read_pickle(file)
    except FileNotFoundError:
        print("No results of previously analyzed datasets were detected.")
        return None
    else: 
        print("Results of previously analyzed datasets were detected.")
        return df
        

def identify_new_files(files, df_ring_data):
    """
    Identify which files have already been analyzed previously. 
    Return list of new files for processing
    """
    new_files = []
    for file in files:
        if file not in df_ring_data['filename'].values:
            new_files.append(file)
    
    n_old = len(files)-len(new_files)
    n_new = len(new_files)
    if n_old == 1:
        print(" {} HDF5 file was previously analyzed.".format(n_old))
    else: 
        print(" {} HDF5 files were previously analyzed.".format(n_old))
        
    if n_new == 1:
        print(" {} HDF5 file is new and will be analyzed.".format(n_new))
    else: 
        print(" {} HDF5 files are new and will be analyzed.".format(n_new))
        
    return new_files
   

def identify_fov_cell_type(df_ring_data, filenames, fov_id_start):
    """
    For each file identify 
    - cell type: sporulating or vegetative cells
    - fov index: fov from which the picks were generated
    Results will be saved in a dataframe with columns:
    'fov_id', 'filename', 'cell_type'
    
    If some files have been analyzed before the current script execution 
    the results were saved in the ring_data file and loaded to df_ring_data.
    (columns: "fov_id", "cell_type", "filename", "group", ... where 
    group are the pick ids.)
    If no prior analysis results exist, then df_ring_data = None. 
    Therefore we can check if a new file belongs to a previously analyzed fov.
    
    """
    dictionary = {}
    filenames_no_cell_type = []
    
    fov_id_counter = fov_id_start + 1
    
    for i, filename in enumerate(filenames):
        # cell type: spor or veg:
        cell_type = np.nan
                
        spor_found = re.search('spor', filename, re.IGNORECASE)
        veg_found = re.search('veg', filename, re.IGNORECASE)
        
        if spor_found and not veg_found:
            cell_type = 'spor'
        elif not spor_found and veg_found:
            cell_type = 'veg'
        elif spor_found and veg_found:
            # consider the string occuring first as cell type determining string
            spor_found_location = spor_found.start()
            veg_found_location = veg_found.start()
            
            if spor_found_location < veg_found_location:
                cell_type = 'spor'
            else:
                cell_type = 'veg'
        else:
            cell_type = np.nan
            filenames_no_cell_type.append(filename)
            

        
        # search if a file from the same fov was already registered:
        if not pd.isnull(cell_type):
            # get substrings of filename that do not contain the cell_type string.
            filename_substrings = re.split(cell_type, filename, flags = re.IGNORECASE)
            
            
            
            # check if an already registered file exists that contains the substrings
            filenames_found = []
            old_or_new_file = [] # True if file from previous run of the script, False if new file.
            for filename2 in dictionary.keys(): # search in files that were already registered.
                contained = all([substring in filename2 for substring in filename_substrings])
                if contained and filename != filename2:
                    filenames_found.append(filename2)
                    old_or_new_file.append(False)
            if isinstance(df_ring_data, pd.DataFrame):
                previously_analyzed_files = np.unique(df_ring_data['filename'])
                for filename2 in previously_analyzed_files:
                    contained = all([substring in filename2 for substring in filename_substrings])
                    if contained and filename != filename2:
                        filenames_found.append(filename2)
                        old_or_new_file.append(True)
            
            # if one other file was found: assign the existing fov index to the newly registred file
            # if no other file was found: assign a new fov index to the newly registered file
            if len(filenames_found) > 1:
                raise Exception('''Files from the same FOV than ''' + filename + ''' where searched. 
                However more than one other file was detected: 
                ''' + '\n'.join(filenames_found))
            elif len(filenames_found) == 1:
                filename_found = filenames_found[0]
                if old_or_new_file[0]: # filename_found is from previous run of the script
                    fov_id = df_ring_data.loc[df_ring_data['filename'] == filename_found, 'fov_id'].iloc[0]
                if not old_or_new_file[0]: # filename_found is also a new file.
                    fov_id = dictionary[filename_found][0]
            else: # No file from the same fov previously registered
                fov_id = fov_id_counter
                fov_id_counter += 1
        
        else:
            fov_id = np.nan
            
        dictionary[filename] = [fov_id, filename, cell_type]
        
    df_results = pd.DataFrame.from_dict(dictionary, orient = 'index', columns = ['fov_id', 'filename', 'cell_type'])
    df_results = df_results.reset_index()

    print()
    print('The cell type (spr or veg) of these files could not be determined')
    print('and thus cannot be used for further analysis:')
    for filename in filenames_no_cell_type:
        print(' -', filename)
        
    return df_results
                
    

def load_data(path):

    try:
        locs, info = io.load_locs(path)
    except io.NoMetadataFileError:
        return None, None, None
    
    try:
        pixelsize = info[1]["Pixelsize"]
    except:  
        print("No pixelsize found in yaml file. Default 130 nm used.")
  
    if hasattr(locs, "x_pick_rot"):

        # convert px to nm
        locs.x *= pixelsize
        locs.y *= pixelsize
        locs.x_pick_rot *= pixelsize
        locs.y_pick_rot *= pixelsize
        
        return locs, info, pixelsize
    else:
        print('x_pick_rot column is missing!')
        return None, None, None


def double_gaus(x,a,x0,sigma, b, x1, sigma1):
    return a*np.exp(-(x-x0)**2/(2*sigma**2)) + b*np.exp(-(x-x1)**2/(2*sigma1**2))

def gaus(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2)) 

def histogram(data, binning=100, column = "y"):
    # histogram
    n, bins = np.histogram(data[column], bins=binning)
    centers = (bins[:-1] + bins[1:]) / 2
    hist_data = [n, centers]
    return hist_data

def fit_peaks(data, p0, binning=100, column = "y", hist_data = None):
    if hist_data is None:
        hist_data = histogram(data, binning, column)
    
    n = hist_data[0]
    centers = hist_data[1]
    
    p_fit, p_cov = optimize.curve_fit(gaus, centers, n, p0=p0)
    p_fit[2] = np.abs(p_fit[2])
    """
    try:
        p_fit, p_cov = optimize.curve_fit(gaus, bins, n, p0=p0)
        p_fit[2] = np.abs(p_fit[2])
    except:
        p_fit = [0,0,0]
    """
    return p_fit, hist_data
    

def find_peaks(data, binning=100, axes="y"):
    
    if axes == "y":
        column = "y_pick_rot"
    elif axes == "x":
        column = "x_pick_rot"
    elif axes == "xyz":
        column = 2
    
    # find peak
    bandwidth = estimate_bandwidth(data[column].reshape(-1, 1), quantile=0.2, n_samples=binning)
    #print("estimated bandwidth: "+str(bandwidth))
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    ms.fit(data[column].reshape(-1, 1))
    labels = ms.labels_
   # print(ms.cluster_centers_[0:2])
    peaks = np.sort(ms.cluster_centers_[0:2], axis=None)
    peak1 = float(ms.cluster_centers_[0])
    estimated_peaks = {0:peak1}
    
    # fit peaks
    p0 = [peak1/2, peak1, 40]
    
    p_fit, hist_data = fit_peaks(data, p0, binning=100, column = column)
    """
    # histogram
    n, bins = np.histogram(data[column], bins=binning)
    bins = bins[1:]
    hist_data = [n, bins]

    # fit peaks
    p0 = [peak1/2, peak1, 40]
    try:
        p_fit, p_cov = optimize.curve_fit(gaus, bins, n, p0=p0)
    except:
        p_fit = [0,0,0]
    """ 
    
    # check order of fitted peaks (peak 1 < peak 2)
    #if p_fit[1] > p_fit[4]:
    #    p_temp = p_fit.copy()
    #    p_fit[0:3]=p_temp[3:6]
    #    p_fit[3:6]=p_temp[0:3]
       
        
    # p_fit: amplitued_1, center_1, width_1, amplitude_2, center_2, width_2
        
    return estimated_peaks, p_fit, hist_data


def plot_peak_dist(data, hist_data, peaks, p_fit, binning=100,cutoff=1, axes="y", title = None, ax=None):
    
    if ax is None:
        ax = plt.gca()
        
    
    if axes == "xyz":
        column == 1
    
    column = axes
        
    peak1 = peaks[0]

    n = hist_data[0]
    bins = hist_data[1]
    
    #fig, ax = plt.subplots(figsize=(9, 5))
    binwidth = (max(bins)-min(bins)) / binning/2
    ax.bar(bins, n, width=binwidth, color=gray)
    xlin = np.linspace(0, data[column].max(), 1000)
    ax.plot(xlin, gaus(xlin,*p_fit[0:3]), c=red, linewidth=2)
    ax.axvline(p_fit[1]-(p_fit[2]*cutoff),c=blue, linewidth=2, linestyle="--")
    ax.axvline(p_fit[1]+(p_fit[2]*cutoff),c=blue, linewidth=2, linestyle="--")
    if title is None:
        ax.set_title("Line profile",loc="left",fontsize=14)
    else:
        ax.set_title(title,loc="left",fontsize=14)
        
    ax.set_xlabel("y (nm)")
    ax.set_ylabel("Counts")  
    ax.text(0.15,
            0.7,
            ("Estimated Peaks:\n"
            "Peak at {:.1f} nm\n"
            "Fitted Peaks:\n"
            "Peak at {:.1f} nm\n"
            "sigma of {:.1f} nm\n").format(peak1, p_fit[1], p_fit[2]),
            horizontalalignment="center",
            verticalalignment="center",
            transform = ax.transAxes,
            fontsize=12)

    return ax


def isolate_ring(data, c, w, axes="y", cutoff=1.0):
    column = axes
    ring_data = data[np.where((data[column]>(c-(cutoff*w))) & (data[column] <(c+(cutoff*w))))]
    return ring_data


def rodrigues_rot(P, n0, n1):
    # adapted from https://meshlogic.github.io/posts/jupyter/curve-fitting/fitting-a-circle-to-cluster-of-3d-points/
    # If P is only 1d array (coords of single point), fix it to be matrix
    if P.ndim == 1:
        P = P[np.newaxis,:]
    
    # Get vector of rotation k and angle theta
    n0 = n0/linalg.norm(n0)
    n1 = n1/linalg.norm(n1)
    k = np.cross(n0,n1)
    k = k/linalg.norm(k)
    theta =np.arccos(np.dot(n0,n1))
    
    # Compute rotated points
    P_rot = np.zeros((len(P),3))
    for i in range(len(P)):
        P_rot[i] = P[i]*np.cos(theta) + np.cross(k,P[i])*np.sin(theta) + k*np.dot(k,P[i])*(1-np.cos(theta))

    return P_rot


def rotate_ring(XYZ): 
    # Fitting plane by SVD for the mean-centered data
    # Eq. of plane is <p,n> + d = 0, where p is a point on plane and n is normal vector
       
    # Normal vector of fitting plane is given by 3rd column in V
    # Note linalg.svd returns V^T, so we need to select 3rd row from V^T
    ring_mean = XYZ.mean(axis=0)
    ring_centered = XYZ - ring_mean
    U,s,V = linalg.svd(ring_centered)
    normal = V[2,:]
    d = -np.dot(ring_mean, normal) 
        
    n0 = normal # new z axes
    n1 = [0,0,1] # old z axes

    ring_rot = rodrigues_rot(ring_centered, n0, n1)
    
    return ring_rot, normal


def plot_3d_ring(ring_rot, dist, ax=None):
    
    ax.scatter(ring_rot.x, ring_rot.y, ring_rot.z-dist/2,c=blue, label="FtsZ", alpha=0.5)
    #ax.scatter(ring_rot[1][:,0], ring_rot[1][:,1], ring_rot[1][:,2]+dist/2,c=orange,label="Mother", alpha=0.5)
    ax.set_xlabel("x (nm)")
    ax.set_ylabel("y (nm)")
    ax.set_zlabel("z (nm)")
    ax.legend(loc="best",labelspacing=0.1)
    #ax.view_init(elev=2., azim=10.)
    ax.view_init(elev=30., azim=10.)
    set_axes_equal_3d(ax)

    return ax
    
    
def set_axes_equal_3d(ax):
    limits = np.array([ax.get_xlim3d(), ax.get_ylim3d(), ax.get_zlim3d()])
    spans = abs(limits[:,0] - limits[:,1])
    centers = np.mean(limits, axis=1)
    radius = 0.5 * max(spans)
    ax.set_xlim3d([centers[0]-radius, centers[0]+radius])
    ax.set_ylim3d([centers[1]-radius, centers[1]+radius])
    ax.set_zlim3d([centers[2]-radius, centers[2]+radius])
    

def angle_between(u, v, n=None):
    if n is None:
        return np.arctan2(np.linalg.norm(np.cross(u,v)), np.dot(u,v))*180/np.pi
    else:
        return np.arctan2(np.dot(n,np.cross(u,v)), np.dot(u,v))*180/np.pi


def calc_R(x, y, xc, yc):
    # adapted from https://gist.github.com/lorenzoriano/6799568
    """ calculate the distance of each 2D points from the center (xc, yc) """
    return np.sqrt((x-xc)**2 + (y-yc)**2)


def f(c, x, y):
    """ calculate the algebraic distance between the data points and the mean circle centered at c=(xc, yc) """
    Ri = calc_R(x, y, *c)
    return Ri - np.median(Ri)


def leastsq_circle(x,y):
    
    x_m = np.median(x)
    y_m = np.median(y)
    center_estimate = x_m, y_m
    center, ier = optimize.leastsq(f, center_estimate, args=(x,y))
    xc, yc = center
    Ri       = calc_R(x, y, *center)
    R        = np.median(Ri)
    residu   = np.sum((Ri - R)**2)
    return xc, yc, Ri, R, residu


def plot_data_circle(x, y, xc, yc, R, residu_norm, id, center=True, ax=None):
    
    if id == 0:
        label = "FtsZ"
        color = blue
    else:
        label = "mother"
        color = orange
    
    if ax is None:
        ax = plt.gca()
    
    if center:
        x -= xc
        y -= yc
        xc = 0
        yc = 0 
        
    #f, ax = plt.subplots(figsize=(5,5))  #figsize=(7, 5.4), dpi=72,
    theta_fit = np.linspace(-np.pi,np.pi, 180)
    x_fit = xc + R*np.cos(theta_fit)
    y_fit = yc + R*np.sin(theta_fit)
    
    # plot fit
    ax.plot(x_fit, y_fit, label="Fitted circle", lw=2, c=red)
    ax.plot([xc], [yc], mec="y", mew=1,  c=red)
    
    # plot data
    ax.scatter(x, y, alpha=0.4,  label="Projected locs", marker=".", c=color)
    
    ax.set_xlabel("y rotated (nm)")
    ax.set_ylabel("z rotated (nm)")
    #ax.axis("equal")
    ax.set_xlim([-800,800])
    ax.set_aspect("equal",adjustable="datalim")
    
    ax.legend(loc="best", labelspacing=0.1)
    """
    ax.set_title("Least squares circle {}\n"
                 "Fit radius: {:.1f} nm\n"
                 "Mean of the |resdiual| values: {:.1f} nm".format(label,R,residu_norm),loc="left",fontsize=14)
    """
    ax.set_title("Least squares circle {}\n"
                 "Fit radius: {:.1f} nm".format(label,R),loc="left",fontsize=14)
    
    
    return 

def polar_coordinate_angle(x_c, y_c, x, y):
    """
    x_center, y_center: center coordinate from ring fit.
    x,y: arrays saving coordinates of localizations.
    """
    xx = x-x_c # x coordinates with x_center as 0
    yy = y-y_c # x coordinates with x_center as 0

    angles = np.degrees(np.arctan2(yy,xx))

    return angles

def check_consistency(ring_par):
    
#    ev = 0

#    amp_1, c_1, w_1 = ring_par[0]
#    amp_2, c_2, w_2 = ring_par[1]

#    if amp_1/amp_2 >= 0.3 and amp_2/amp_1 >= 0.3:
#        ev +=1 

#    if abs(c_1-c_2) > 30 and abs(c_1-c_2) < 300:
#        ev +=1

#    if w_1/w_2 >= 0.3 and w_2/w_1 >=0.3:
#        ev +=1
        
    return (True)

def plot_cum_exp(pooled_locs, fit_result_len, fit_result_dark, id, ax=None):
    
    if id == 0:
        color = blue
        label = 'FtsZ'
    if id == 1:
        color = orange
        label = 'mother'

    if ax is None:
        ax = plt.gca()
    
    data = pooled_locs.dark
    data.sort()
    y = np.arange(1, len(data) + 1)
       
    a = fit_result_dark.best_values["a"]
    t = fit_result_dark.best_values["t"]
    c = fit_result_dark.best_values["c"]

    ax.set_title(
        "Dark time (cumulative) {}\n"
        r"$Fit: {:.2f}\cdot(1-exp(x/{:.2f}))+{:.2f}$".format(label, a, t, c),loc="left",fontsize=14)
    data = pooled_locs.dark
    data.sort()
    y = np.arange(1, len(data) + 1)

    ax.semilogx(data, y, c=color, label="Data")
    ax.semilogx(data, fit_result_dark.best_fit, c=red, label="Fit")
    ax.legend(loc="best")
    ax.set_xlabel("Duration (frames)")
    ax.set_ylabel("Frequency")
    
    return ax


def save_ring_locs(locs, info, path, fov_id, cell_type, pick, id, link=False, filename = ''):
    
    if link:
        ending = "_link.hdf5"
    else:
        ending = ".hdf5"
    
    if filename != '':
        ending = filename + ending
       
    locs.x /= 130
    locs.y /= 130
    
    if np.isnan(id):
        locs_name =  "fov_{}_{}_pick_{}{}".format(fov_id, cell_type, pick, ending)
    else:
        locs_name =  "fov_{}_{}_pick_{}_ring_{}{}".format(fov_id, cell_type, pick, id, ending)
    locs_path = os.path.join(path,"ring_locs")
    locs_path_name = os.path.join(locs_path, locs_name)
    
    if not os.path.isdir(locs_path):
        os.makedirs(locs_path, exist_ok=True)

    
    io.save_locs(locs_path_name, locs, info)

def export_pick_img(locs, path, fov_id, cell_type, pick, id, link=False):
    
    if link:
        ending = "_link.png"
    else:
        ending = ".png"
    
    pixelsize = 130

    export_locs = locs.copy()
    
    export_locs.x /= pixelsize
    export_locs.y /= pixelsize
    
    x_min = np.min(export_locs.x)    
    x_max = np.max(export_locs.x)
    y_min = np.min(export_locs.y)
    y_max = np.max(export_locs.y)

    viewport =  (y_min, x_min), (y_max, x_max)
    oversampling = 50
    len_x, image = render(export_locs, viewport = viewport, oversampling=oversampling, blur_method="smooth")
    
    img_name = "fov_{}_{}_pick_{}_ring_{}{}".format(fov_id, cell_type, pick, id, ending)
    img_path = os.path.join(path,"ring_images")
    img_path_name = os.path.join(img_path,img_name)
    
    if not os.path.isdir(img_path):
        os.makedirs(img_path)
    
    plt.imsave(img_path_name, image, cmap="hot")


def plot_locs_z_colormap(data, axes, title, fig, ax):

    # Generate data...
    if axes == "y_pick_rot":
        x = data.y_pick_rot
        y = data.x_pick_rot
        z = data.z
        ax.set_ylabel("x (nm)")
        ax.set_xlabel("y (nm)") 
    elif axes == "y":
        x = data.y
        y = data.x
        z = data.z
        ax.set_ylabel("x (nm)")
        ax.set_xlabel("y (nm)") 
    elif axes == "x_pick_rot":
        x = data.x_pick_rot
        y = data.y_pick_rot
        z = data.z
        ax.set_xlabel("x (nm)")
        ax.set_ylabel("y (nm)")
    elif axes == "x":
        x = data.x
        y = data.y
        z = data.z
        ax.set_xlabel("x (nm)")
        ax.set_ylabel("y (nm)")  
    
    sc = ax.scatter(x, y, s = 10, c=z, cmap='jet')
    fig.colorbar(sc, ax = ax)
    
    ax.set_aspect('equal', adjustable='datalim')

    ax.set_title(title,loc="left",fontsize=14)
    
    return ax

def plot_locs_overlay(data1, data2, axes, ax, label1 = '', label2 = '', title = ''):

    # Generate data...
    if axes == "y_pick_rot":
        x1 = data1.y_pick_rot
        y1 = data1.x_pick_rot
        z1 = data1.z
        
        x2 = data2.y_pick_rot
        y2 = data2.x_pick_rot
        z2 = data2.z
        
        ax.set_ylabel("x (nm)")
        ax.set_xlabel("y (nm)")
    elif axes == "y":
        x1 = data1.y
        y1 = data1.x
        z1 = data1.z
        
        x2 = data2.y
        y2 = data2.x
        z2 = data2.z
        
        ax.set_ylabel("x (nm)")
        ax.set_xlabel("y (nm)")
    elif axes == "x_pick_rot":
        x1 = data1.x_pick_rot
        y1 = data1.y_pick_rot
        z1 = data1.z
        
        x2 = data2.x_pick_rot
        y2 = data2.y_pick_rot
        z2 = data2.z
    elif axes == "x":
        x1 = data1.x
        y1 = data1.y
        z1 = data1.z
        
        x2 = data2.x
        y2 = data2.y
        z2 = data2.z
    
        ax.set_xlabel("x (nm)")
        ax.set_ylabel("y (nm)")  
    
    ax.scatter(x1, y1, s = 10, c='red', label = label1)
    ax.scatter(x2, y2, s = 10, c='blue', label = label2)
    
    ax.set_aspect('equal', adjustable='datalim')
    
    ax.set_title(title,loc="left",fontsize=14)
    
    ax.legend()
    
    return ax

print('done')


done


## Load data

In [4]:
#path = gui_fname()
path = r'W:\users\reinhardt\z.software\Git\spor-PAINT\dev_sr\spor-paint\FtsZ\FtsZ_picks_all\small_dataset'
filenames_all = load_files(path)


plotting = True
max_dist = 130 #nanometer
max_dark_time = 15 #frames
binning = 50 # binning for peak histogram




3 HDF5 files found.


In [5]:
# Check if some of the found hdf5 files were already analyzed?
# If yes, open ring_data dataframe with previous results.
df_ring_data = load_ring_data_df(path, "ring_data.pkl")


# Identify which files have not yet been analyzed.
if df_ring_data is not None:
    filenames = identify_new_files(filenames_all, df_ring_data)
    fov_id_start = df_ring_data['fov_id'].max()
else:
    filenames = filenames_all
    fov_id_start = 0

# Create a dictionary that saves which file was taken from which FOV and which cell types are contained (spor or veg)
# {filename_1: (FOV_id, 'spor'), filename_2: (FOV_id, 'veg'), ...}
df_fov_file_assign = identify_fov_cell_type(df_ring_data, filenames, fov_id_start)


No results of previously analyzed datasets were detected.

The cell type (spr or veg) of these files could not be determined
and thus cannot be used for further analysis:
 - 1FS_fov2_200pM_r3_23mW561_3dDP_1_1_drift_picked.hdf5


In [6]:
for fov_id in range(int(fov_id_start)+1, int(df_fov_file_assign['fov_id'].max())+1):
    
    print('FOV ID:', fov_id)
    files_fov_id = df_fov_file_assign.loc[df_fov_file_assign['fov_id'] == fov_id]

    
    print('  spor : ', end = '')

    spor_name = files_fov_id.loc[files_fov_id['cell_type'] == 'spor']['filename']
    if spor_name.empty:
        print('--')
    else:
        print(spor_name.iloc[0])

        
    print('  veg  : ', end = '')

    veg_name = files_fov_id.loc[files_fov_id['cell_type'] == 'veg']['filename']
    if veg_name.empty:
        print('--')
    else:
        print(veg_name.iloc[0])


FOV ID: 1
  spor : SPOR_FtsZ_221006_spor_KCB324_rab5xR2_GFP5xR1_fov2_FtsZ_1_aligned_picked.hdf5
  veg  : VEG_FtsZ_221006_spor_KCB324_rab5xR2_GFP5xR1_fov2_FtsZ_1_aligned_picked.hdf5


## Main loop

In [ ]:
print('start')
# initialize containers
ring_data = []
ring_locs = {}
ring_locs_linked = {}
ring_kinetics = {}
ring_kinetics_fit = {}
ring_radii = {}
ring_rot_xy = {}
ring_locs_rot_xy = {}
ring_rot_yz = {}
ring_locs_rot_yz = {}
pick_rot_xy = {}
pick_locs_rot_xy = {}
pick_rot_yz = {}
pick_locs_rot_yz = {}
ring_locs_rot_yz_update = {}
ring_locs_rot_yz_update_0 = {}
ring_locs_rot_xy_update = {}
ring_locs_rot_xy_update_0 = {}
ring_angles = {}
center = {}

rings_excluded = []
ring_n_events = {}
circle_plots = {}


analysis_folder = os.path.join(path, "analysis")
# Tilt correction folder:
# Contains original pick, the selected ring locs based on the first fit and the tilt corrected ring.
tilt_correction_folder = os.path.join(analysis_folder, "tilt_correction")
tilt_correction_plot_folder = os.path.join(tilt_correction_folder, "plots")
analysis_folder_excluded = os.path.join(analysis_folder, "excluded")
ring_locs_folder = os.path.join(analysis_folder, "ring_locs")

# image export settings
img_format = ".png"
dpi = 100

# cutoff is mutliplied to the sigma of the peak fit for the seperation of the two rings
# smaller cutoff means smaller ring sections
cutoff = 1.5

#prepare analysis folder
if not os.path.isdir(analysis_folder):
    os.makedirs(analysis_folder)
if not os.path.isdir(tilt_correction_folder):
    os.makedirs(tilt_correction_folder)
if not os.path.isdir(tilt_correction_plot_folder):
    os.makedirs(tilt_correction_plot_folder)
if not os.path.isdir(analysis_folder_excluded):
    os.makedirs(analysis_folder_excluded)
if not os.path.isdir(ring_locs_folder):
    os.makedirs(ring_locs_folder)

files_no_x_pick_rot = []
    
#iterate over locs in directory:
for filename in tqdm(filenames, desc="Processing files"):
    print()
    print(filename)
    
    fov_id = df_fov_file_assign.loc[df_fov_file_assign['filename'] == filename]['fov_id'].iloc[0]
    cell_type = df_fov_file_assign.loc[df_fov_file_assign['filename'] == filename]['cell_type'].iloc[0]

    
    if np.isnan(fov_id):
        print('FOV ID is nan.')
        continue
    fov_id = int(fov_id)
    
    #load locs and convert distances from px to nm (Attention!)
    locs, info, pixelsize = load_data(os.path.join(path,filename))
    
    if locs is None:
        files_no_x_pick_rot.append(filename)
        print("File {} not loaded.".format(filename))
        continue
    
    # iterate over picks in a file
    for pick in tqdm(np.unique(locs.group), desc="Processing picks"):
     
        # select locs from pick
        pick_locs = locs[locs.group == pick]

        ################
        # (1) estimate the postition of the two rings using a histogram along the pick direction
        #  a ring should yield a gaussian peak (2D projection in XY)
        ################
        
        estimated_peaks, r_par, hist_data = find_peaks(pick_locs, binning=binning, axes="y")
        # estimated peaks are rough estimates of peak position from MeanShift analysis 
        # used to initialize guassian fits
        # r_par yields an array containing: [amplitued_1, center_1, width_1]

        # seperate ring parameter to the corresponding ring
        ring_parameter = [] # to work as the double band script
        ring_parameter.append(r_par)

        if check_consistency(ring_parameter):
        # no checks in the single band case
            
            # set up plot illustrating tilt correction with gridspec
            fig = plt.figure(figsize=(23, 14), constrained_layout=True)
            gs = fig.add_gridspec(2,3)
            fig.suptitle(("FOV {}, {}, Pick {} - Tilt correction\n"
                          "File: {}").format(fov_id, cell_type, pick, filename), 
                         fontsize=16, 
                         ha="center")
            
            # plot histogram for locs in pick before tilt correction
            ax1 = fig.add_subplot(gs[0, 0])
           
            plot_peak_dist(pick_locs,
                           hist_data,
                           estimated_peaks,
                           r_par,
                           binning=binning,
                           cutoff=cutoff,
                           axes="y_pick_rot",
                           title="Line profile: before tilt correction",
                           ax=ax1)
            
            # plot scatter plot for locs in pick with z colorcode
            ax2 = fig.add_subplot(gs[0, 1])
            plot_locs_z_colormap(pick_locs, 
                                 'y_pick_rot', 
                                 title = 'Pick localizations: before tilt correction',
                                 fig = fig, 
                                 ax = ax2)
            ax2.set_xlim(ax1.get_xlim())
            


        
            ################
            # (2) isolate the locs from every ring
            # fitted center +- peak-to-peak distance/2
            ################
            
            
            for id, ring in enumerate(ring_parameter):
    
                # isolate the locs from every ring
                r_locs = isolate_ring(pick_locs, c=ring[1], w=ring[2], cutoff=cutoff, axes="y_pick_rot")
                ring_locs[id] = r_locs
                
                """
                # link the isolated locs
                r_locs_linked = link(r_locs, info, max_dist, max_dark_time)
                ring_locs_linked[id] = r_locs_linked
                
                # extract number of locs
                ring_n_events[id] = len(r_locs_linked)
                """
                
                # plot locs that are in the cutoff region vs the ones outside of it.
                ax3 = fig.add_subplot(gs[0, 2])
                plot_locs_overlay(pick_locs, 
                                     ring_locs[id],
                                     'y_pick_rot',
                                     ax = ax3,
                                     label1 = 'locs in pick',
                                     label2 = 'locs within ' + str(cutoff) +' x sigma',
                                     title = 'Pick and ring localizations: before tilt correction')
                ax3.set_xlim(ax1.get_xlim())
            
            
              
                        
            ################
            # (3b) Determine tilt of rings and save tilt corrected localizations
            ################
            
            
            #ax3 = fig.add_subplot(gs[1, 0])
            #ax5 = fig.add_subplot(gs[2, 0])
            normal = {}
            for id, rings in ring_locs.items():
                
                z = np.abs(stats.zscore(rings.z))
                
                # prepare XYZ coordinates
                XYZ = np.array([rings.x, rings.y, rings.z]).transpose()
                XYZ_mean = XYZ.mean(axis=0)
                
                # rotate coordinates into the plane of the ring. ring should be now in X-Y
                ring_rot_xy[id], normal[id] = rotate_ring(XYZ)
                
                
                # rotate ring coordinates into the plane of the ring. ring should be now in Y-Z
                ring_rot_yz[id] = rodrigues_rot(XYZ - XYZ.mean(axis=0), normal[id], [1,0,0])
                # create rec_array with rotated locs
                ring_locs_rot_yz[id] = ring_locs[id].copy()
                ring_locs_rot_yz[id].x = ring_rot_yz[id][:,0]+info[0]['Width']*130/2
                ring_locs_rot_yz[id].y = ring_rot_yz[id][:,1]+info[0]['Height']*130/2
                ring_locs_rot_yz[id].z = ring_rot_yz[id][:,2]
                           
            # calculate the angle between coverslip and spore or mother ring
            coverslip_ring_angle = angle_between(normal[0],[0,0,1])
            
            
            # rotate pick coordinates to be in X-Y plane or in Y-Z plane
            # prepare XYZ coordinates
            XYZ_pick = np.array([pick_locs.x, pick_locs.y, pick_locs.z]).transpose()
            
            """
            # rotate pick coordinates into the plane of the ring. ring should be now in X-Y
            pick_rot_xy = rodrigues_rot(XYZ_pick - XYZ_mean, normal[id], [0,0,1])
            # create rec_array with rotated locs
            pick_locs_rot_xy = pick_locs.copy()
            pick_locs_rot_xy.x = pick_rot_xy[:,0]+info[0]['Width']*130/2
            pick_locs_rot_xy.y = pick_rot_xy[:,1]+info[0]['Height']*130/2
            pick_locs_rot_xy.z = pick_rot_xy[:,2]
            """
            
            # rotate pick coordinates into the plane of the ring. ring should be now in Y-Z
            pick_rot_yz = rodrigues_rot(XYZ_pick - XYZ_mean, normal[id], [1,0,0])
            # create rec_array with rotated locs
            pick_locs_rot_yz = pick_locs.copy()
            pick_locs_rot_yz.x = pick_rot_yz[:,0]+info[0]['Width']*130/2
            pick_locs_rot_yz.y = pick_rot_yz[:,1]+info[0]['Height']*130/2
            pick_locs_rot_yz.z = pick_rot_yz[:,2]
            
            
            
            ################
            # (4) repeat Gaussian fit on tilt corrected data and isolate rings 
            ################
                      
            # move locs to (0,0) for the fit
            pick_locs_rot_yz_0 = pick_locs_rot_yz.copy()
            dx = pick_locs_rot_yz_0.x.min()
            dy = pick_locs_rot_yz_0.y.min()
            pick_locs_rot_yz_0.x = pick_locs_rot_yz_0.x - dx
            pick_locs_rot_yz_0.y = pick_locs_rot_yz_0.y - dy
            
            # fit pick locs after alignment to y-z axis
            hist_data = histogram(pick_locs_rot_yz_0, binning=100, column = "x")
            p0 = [r_par[0], r_par[1], r_par[2]]
            n = hist_data[0]
            bins = hist_data[1]
            p0[1] = bins[np.argmax(n)]
            r_par_yz_aligned, hist_data_yz_aligned = fit_peaks(pick_locs_rot_yz_0, p0 = p0, binning=100, column = "x", hist_data = hist_data)
            

            # seperate ring parameter to the corresponding ring
            ring_parameter_yz_aligned = [] # to work as the double band script
            ring_parameter_yz_aligned.append(r_par_yz_aligned)

            
            
            # plot histogram for locs in pick after tilt correction
            ax4 = fig.add_subplot(gs[1, 0])
           

            plot_peak_dist(pick_locs_rot_yz_0,
                           hist_data_yz_aligned,
                           {0: r_par[1]},
                           r_par_yz_aligned,
                           binning=binning,
                           cutoff=cutoff,
                           axes="x",
                           title="Line profile: after tilt correction",
                           ax=ax4)
            ax4.set_xlim(ax1.get_xlim())
            
            
            # plot scatter plot for locs in pick with z colorcode
            ax5 = fig.add_subplot(gs[1, 1])
            plot_locs_z_colormap(pick_locs_rot_yz_0, 
                                 'x', 
                                 title = 'Pick localizations: after tilt correction',
                                 fig = fig, 
                                 ax = ax5)
            ax5.set_xlim(ax1.get_xlim())
            
            
            ################
            # (4.1) isolate the locs from every ring in the tilt corrected data
            # fitted center +- peak-to-peak distance/2
            ################
            
            
            for id, ring in enumerate(ring_parameter_yz_aligned):
    
                #print("ring {} with limits {} & {}".format(id,ring[1]-(ring[2]*cutoff),ring[1]+(ring[2]*cutoff)))
                # isolate the locs from every ring
                r_locs = isolate_ring(pick_locs_rot_yz_0, c=ring[1], w=ring[2], cutoff=cutoff, axes="x")
                ring_locs_rot_yz_update_0[id] = r_locs.copy()
                
                # get flat version by rotating 90 degrees around y axis
                ring_locs_rot_xy_update_0[id] = r_locs.copy()
                ring_locs_rot_xy_update_0[id].x = r_locs.z
                ring_locs_rot_xy_update_0[id].z = - r_locs.x
                
                
                # plot locs that are in the cutoff region vs the ones outside of it.
                ax6 = fig.add_subplot(gs[1, 2])
                plot_locs_overlay(pick_locs_rot_yz_0, 
                                     r_locs,
                                     'x',
                                     ax = ax6,
                                     label1 = 'locs in pick',
                                     label2 = 'locs within ' + str(cutoff) +' x sigma',
                                     title = 'Pick and ring localizations: after tilt correction')
                ax6.set_xlim(ax1.get_xlim())
                
                
                # move to center of FOV: 
                ring_locs_rot_yz_update[id] = r_locs.copy()
                ring_locs_rot_yz_update[id].x = ring_locs_rot_yz_update[id].x + dx
                ring_locs_rot_yz_update[id].y = ring_locs_rot_yz_update[id].y + dy
                
                ring_locs_rot_xy_update[id] = ring_locs_rot_xy_update_0[id].copy()
                ring_locs_rot_xy_update[id].x = ring_locs_rot_xy_update_0[id].x + dx
                ring_locs_rot_xy_update[id].y = ring_locs_rot_xy_update_0[id].y + dy

                
                
            
            
            ################
            # (5) Ring analysis
            ################
            
            
            # set up plot illustrating analysis parameters
            fig2 = plt.figure(figsize=(24, 14), constrained_layout=True)
            gs2 = fig2.add_gridspec(2,2)
            fig2.suptitle(("FOV {}, {}, Pick {} - Ring analysis\n"
                          "File: {}").format(fov_id, cell_type, pick, filename), 
                         fontsize=16, 
                         ha="center")
            
            ax2_1 = fig2.add_subplot(gs2[0, 0])
            # plot histogram for locs in pick after tilt correction
            plot_peak_dist(pick_locs_rot_yz_0,
                           hist_data_yz_aligned,
                           {0:r_par[1]},
                           r_par_yz_aligned,
                           binning=binning,
                           cutoff=cutoff,
                           axes="x",
                           title="Line profile: after tilt correction",
                           ax=ax2_1)
            #ax2_1.set_xlim(ax1.get_xlim())
            
            
            # plot 3D ring
            ax2_2 = fig2.add_subplot(gs2[0, 1], projection="3d")
            plot_3d_ring(ring_locs_rot_yz_update_0[0],0, ax=ax2_2)
                        
            # Link locs
            for id, ring in ring_locs_rot_yz_update_0.items():
                
                
                # link the isolated locs
                r_locs_linked = link(ring, info, max_dist, max_dark_time)
                ring_locs_linked[id] = r_locs_linked
                
                # extract number of events
                ring_n_events[id] = len(r_locs_linked)

            
            
            # fit ring
            ax2_3 = fig2.add_subplot(gs2[1, 0])
            for id, ring in ring_locs_rot_yz_update_0.items():
                 


                # fit 2D ring into the YZ ring data
                y_center, z_center, radii, Radius, residu = leastsq_circle(ring.y, ring.z)
                residu_vector = radii - Radius
                
                # append radius of each localization (distance to ring center) to hdf5 file. 
                ring_locs_rot_yz_update[id] = append_to_rec(ring_locs_rot_yz_update[id], radii, 'radius')
                ring_locs_rot_xy_update[id] = append_to_rec(ring_locs_rot_xy_update[id], radii, 'radius')
                
                # calculate angle in polar coordinates for each localization                
                angles = polar_coordinate_angle(y_center, z_center, ring.y, ring.z)
                # append angle of each localization (polar coordinates with (0,0) being the ring center) to hdf5 file. 
                ring_locs_rot_yz_update[id] = append_to_rec(ring_locs_rot_yz_update[id], angles, 'angle')
                ring_locs_rot_xy_update[id] = append_to_rec(ring_locs_rot_xy_update[id], angles, 'angle')
                
                    
                    
                
                """
                c = rodrigues_rot(np.array([y_center, z_center,0]), [0,0,1], normal[id]) + XYZ.mean(axis=0)
                center[id] = c.flatten()
                """
                
                # extract radii
                ring_radii[id] = Radius
                
                # plot
                if id == 0:
                    ax_circ = ax2_3

                resdiu_absolute_avg = np.mean(np.abs(residu_vector))
                
                plot_data_circle(ring.y,
                                 ring.z,
                                 y_center,
                                 z_center,
                                 Radius,
                                 resdiu_absolute_avg,
                                 id,
                                 center=True,
                                 ax=ax_circ)
                
                # plot residuals
                """
                ax2_4 = fig2.add_subplot(gs2[1, 1])
                
                n, bins = np.histogram(residu_vector, bins = binning) #, bins=binning
                centers = (bins[:-1] + bins[1:]) / 2
                binwidth = bins[1]-bins[0]
                ax2_4.bar(centers, n, width=binwidth, color=gray)
                title = "Distribution of residual values"
                ax2_4.set_title(title,loc="left",fontsize=14)
                ax2_4.set_xlabel("Residuals (nm)")
                ax2_4.set_ylabel("Counts")
                """


                
            ################
            # (5.1) qPAINT analysis
            ################
            
            ax2_5 = fig2.add_subplot(gs2[1, 1])
            
            for id, rings_linked in ring_locs_linked.items():

                    # compute kinetics
                    kinetics = compute_dark_times(rings_linked)
                    #print('kinetics', kinetics)

                    # compute mean bright and dark times
                    ring_kinetics[id] = {"bright":np.nanmean(kinetics.len),
                                         "dark":np.nanmean(kinetics.dark)}
                    
                    fit_result_len = fit_cum_exp(kinetics.len)
                    fit_result_dark = fit_cum_exp(kinetics.dark)
                
                    if id == 0:
                        ax_fit=ax2_5

                    
                    plot_cum_exp(kinetics, fit_result_len, fit_result_dark, id, ax=ax_fit)
                    
                    ring_kinetics_fit[id] = {"bright":np.nanmean(fit_result_len.best_values["t"]),
                                             "dark":np.nanmean(fit_result_dark.best_values["t"])}
            
            
            
            ################
            # (5.2) append data into large array
            ################
            
            ring_data.append([fov_id, #running file index
                              cell_type, # veg or spore
                              filename, #filename
                              pick, #pick number
                              residu, #residual of ring fit
                              
                              ring_radii[0], #fitted radius of spore (nm)
                              coverslip_ring_angle, #anle between spore ring and coverslip (°)
                              ring_parameter_yz_aligned[0][2], #FWHM
                              ring_n_events[0], #number of locs in spore ring
                              ring_kinetics[0]["bright"], #mean bright time of spore ring (frames)
                              ring_kinetics[0]["dark"], #mean dark time of spore ring (frames)
                              ring_kinetics_fit[0]["bright"], #cumulative mean bright times of spore (frames)
                              ring_kinetics_fit[0]["dark"] #cumulative mean dark times of spore (frames)
                             ])
            
            ################
            # (6) save plots
            ################
            
            # tilt correction
            img_fname = "fov_{}_{}_pick_{}_tilt_corr".format(fov_id, cell_type, pick)
            img_name = os.path.join(tilt_correction_plot_folder, img_fname)
            fig.savefig(img_name+img_format, dpi=dpi, format="png")
            plt.close(fig)
            
            # ring analysis
            img_fname = "fov_{}_{}_pick_{}".format(fov_id, cell_type, pick)
            img_name = os.path.join(analysis_folder, img_fname)
            fig2.savefig(img_name+img_format, dpi=dpi, format="png")
            plt.close(fig2)
            
            
            
            
            ################
            # (7) export ring locs
            ################


            # before tilt correction
            # pick locs
            save_ring_locs(pick_locs, info, tilt_correction_folder, fov_id, cell_type, pick, np.nan, link=False, filename = '')
            # ring locs (locs within cutoff)
            for id, rings in ring_locs.items():
                save_ring_locs(rings, info, tilt_correction_folder, fov_id, cell_type, pick, id, link=False)
            # after tilt correction
            # ring locs (locs that were within cutoff)
            for id, rings in ring_locs_rot_yz.items():
                save_ring_locs(rings, info, tilt_correction_folder, fov_id, cell_type, pick, id, link=False, filename = '_rot_yz')
            
            # after tilt correction
            # pick locs
            save_ring_locs(pick_locs_rot_yz, info, analysis_folder, fov_id, cell_type, pick, np.nan, link=False, filename = '_rot_yz')
            # ring locs (locs within cutoff around tilt corrected fit)
            for id, rings in ring_locs_rot_yz_update.items():
                save_ring_locs(rings, info, analysis_folder, fov_id, cell_type, pick, id, link=False, filename = '_rot_yz_update')
            # save ring locs in flat orientation instead of standing orientation
            for id, rings in ring_locs_rot_xy_update.items():
                save_ring_locs(rings, info, analysis_folder, fov_id, cell_type, pick, id, link=False, filename = '_rot_xy_update')
            
            #save_ring_locs(pick_locs_rot_xy, info, analysis_folder, fov_id, cell_type, pick, np.nan, link=False, filename = '_rot_xy')

            
        else:
            fig, ax = plt.subplots(1, 1, figsize=(7, 5), constrained_layout=True)
            fig.suptitle(("FOV {} {}, Pick {} - Ring analysis\n"
                          "File: {}").format(fov_id, cell_type, pick, filename), fontsize=16, ha="center")
            
            plot_peak_dist(pick_locs, hist_data, estimated_peaks, r_par, binning=binning, axes="y_pick_rot", ax=ax)
            img_fname = "fov_{}_{}_pick_{}".format(fov_id, cell_type, pick)
            img_name = os.path.join(analysis_folder_excluded, img_fname)
            plt.savefig(img_name+img_format, format="png")
            plt.close(fig)
            rings_excluded.append(pick)

print()
print('The following files could not be loaded.')
print('Probably they miss the x_pick_rot column:')
for filename in files_no_x_pick_rot:
    print(' -', filename)

print()
print("Calculation finished.")
print("Total: {} rings analysed, {} excluded.".format(len(ring_data), len(rings_excluded)))


start


Processing files:   0%|          | 0/3 [00:00<?, ?it/s]


VEG_FtsZ_221006_spor_KCB324_rab5xR2_GFP5xR1_fov2_FtsZ_1_aligned_picked.hdf5


Processing picks:   0%|          | 0/9 [00:00<?, ?it/s]

## Save analysis data

In [8]:
# todo: add normal vector
df_ring_data_add = pd.DataFrame(ring_data, columns=["fov_id",
                                                "cell_type",
                                                "filename",
                                                "group",
                                                "residual",
                                                "radius",
                                                "angle_between_ring_and_coverslip",
                                                "Sigma",
                                                "n_events",
                                                "mean_bright",
                                                "mean_dark",
                                                "fit_bright",
                                                "fit_dark",
                                               ])
if df_ring_data is not None:
    df_ring_data_save = pd.concat([df_ring_data, df_ring_data_add], ignore_index = True)
else:
    df_ring_data_save = df_ring_data_add
    
df_ring_data_save.to_csv("ring_data.csv")        
# Save dataframe with cell means for easy loading of data for postprocessing
df_ring_data_save.to_pickle("ring_data.pkl")


print("Data saved to CSV file in locs folder.")

Data saved to CSV file in locs folder.


{0: rec.array([(   55, 286.94257, 446.89218, 3502.5276, 1.8671156, 0.9978297 , 80.892395, 0.06263571, 0.02721411, 0.46557692, 11063.124 , 186.02939, 0.01958825,  59.230602 , 151.27956, 0),
           (   56, 286.4608 , 446.71573, 7612.213 , 1.7522103, 0.96453935, 88.88461 , 0.03346481, 0.01662372, 0.44952992, 23734.055 , 141.31807, 0.03546379,   5.4026055, 190.66795, 0),
           (   57, 286.48608, 446.76953, 8011.8667, 1.8225521, 0.9628848 , 85.75826 , 0.03383575, 0.01607805, 0.47168326, 24546.97  , 167.2068 , 0.03656639,  13.04658  , 191.8112 , 0),
           ...,
           (14809, 287.55243, 448.2808 , 5361.7026, 2.2095728, 1.1128038 , 61.288536, 0.05447531, 0.02306123, 0.4963715 , 14426.991 , 257.57446, 0.02185672, 253.4097   , 185.4469 , 0),
           (14810, 287.46918, 448.23654, 5594.4595, 2.0749352, 1.0684788 , 60.240303, 0.04823198, 0.02149085, 0.48505437, 15630.1875, 239.2464 , 0.00802789, 242.6199   , 191.26102, 0),
           (14811, 287.41733, 448.29565, 5277.426 , 2.0